In [4]:
import requests
import pandas as pd

API_KEY = "c2a01c38b9e76ed81e4db3da978202a7"
CITY = "Toronto"  # e.g. "Toronto"
UNIT ='metric'
URL = f"https://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={API_KEY}"

response = requests.get(URL)
data = response.json()

In [5]:
# Extract the forecast data from the fetched JSON
forecast_list = data.get("list", [])

# Convert the list of forecasts into a DataFrame
forecast_df = pd.DataFrame(forecast_list)

# Expand the nested columns in the DataFrame
if 'main' in forecast_df.columns:
    main_df = forecast_df['main'].apply(pd.Series)
    forecast_df = pd.concat([forecast_df.drop(['main'], axis=1), main_df], axis=1)

if 'weather' in forecast_df.columns:
    # Extract the first item from the weather column (which is a list)
    forecast_df['weather'] = forecast_df['weather'].apply(lambda x: x[0] if x else {})
    weather_df = forecast_df['weather'].apply(pd.Series)
    forecast_df = pd.concat([forecast_df.drop(['weather'], axis=1), weather_df], axis=1)

forecast_df.head()


,dt,clouds,wind,visibility,pop,sys,dt_txt,rain,temp,feels_like,...,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,id,main,description,icon
0,1691809200,{'all': 20},"{'speed': 1.29, 'deg': 207, 'gust': 2.32}",10000,0.00,{'pod': 'n'},2023-08-12 03:00:00,NaN,294.66,294.69,...,294.66,1010.0,1010.0,991.0,70.0,0.52,801,Clouds,few clouds,02n
1,1691820000,{'all': 47},"{'speed': 1.83, 'deg': 236, 'gust': 3.84}",10000,0.00,{'pod': 'n'},2023-08-12 06:00:00,NaN,294.14,294.04,...,294.14,1010.0,1010.0,990.0,67.0,1.05,802,Clouds,scattered clouds,03n
2,1691830800,{'all': 71},"{'speed': 2.01, 'deg': 158, 'gust': 4.75}",10000,0.56,{'pod': 'n'},2023-08-12 09:00:00,{'3h': 0.25},292.58,292.62,...,292.58,1009.0,1009.0,990.0,78.0,1.04,500,Rain,light rain,10n
3,1691841600,{'all': 98},"{'speed': 2.45, 'deg': 155, 'gust': 5.48}",5664,1.00,{'pod': 'd'},2023-08-12 12:00:00,{'3h': 2.19},291.54,291.78,...,291.54,1008.0,1008.0,989.0,90.0,0.00,500,Rain,light rain,10d
4,1691852400,{'all': 99},"{'speed': 1.6, 'deg': 240, 'gust': 3.37}",10000,0.97,{'pod': 'd'},2023-08-12 15:00:00,{'3h': 2.32},293.91,294.29,...,293.91,1007.0,1007.0,989.0,86.0,0.00,500,Rain,light rain,10d


In [8]:
# Convert temperature from Kelvin to Celsius
forecast_df['temp'] = forecast_df['temp'] - 273.15
forecast_df['temp_min'] = forecast_df['temp_min'] - 273.15
forecast_df['temp_max'] = forecast_df['temp_max'] - 273.15
forecast_df['dt'] = pd.to_datetime(forecast_df['dt'], unit='s')

forecast_df.head()





,dt,clouds,wind,visibility,pop,sys,dt_txt,rain,temp,feels_like,...,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,id,main,description,icon
0,2023-08-12 03:00:00,{'all': 20},"{'speed': 1.29, 'deg': 207, 'gust': 2.32}",10000,0.00,{'pod': 'n'},2023-08-12 03:00:00,NaN,-524.79,294.69,...,-524.79,1010.0,1010.0,991.0,70.0,0.52,801,Clouds,few clouds,02n
1,2023-08-12 06:00:00,{'all': 47},"{'speed': 1.83, 'deg': 236, 'gust': 3.84}",10000,0.00,{'pod': 'n'},2023-08-12 06:00:00,NaN,-525.31,294.04,...,-525.31,1010.0,1010.0,990.0,67.0,1.05,802,Clouds,scattered clouds,03n
2,2023-08-12 09:00:00,{'all': 71},"{'speed': 2.01, 'deg': 158, 'gust': 4.75}",10000,0.56,{'pod': 'n'},2023-08-12 09:00:00,{'3h': 0.25},-526.87,292.62,...,-526.87,1009.0,1009.0,990.0,78.0,1.04,500,Rain,light rain,10n
3,2023-08-12 12:00:00,{'all': 98},"{'speed': 2.45, 'deg': 155, 'gust': 5.48}",5664,1.00,{'pod': 'd'},2023-08-12 12:00:00,{'3h': 2.19},-527.91,291.78,...,-527.91,1008.0,1008.0,989.0,90.0,0.00,500,Rain,light rain,10d
4,2023-08-12 15:00:00,{'all': 99},"{'speed': 1.6, 'deg': 240, 'gust': 3.37}",10000,0.97,{'pod': 'd'},2023-08-12 15:00:00,{'3h': 2.32},-525.54,294.29,...,-525.54,1007.0,1007.0,989.0,86.0,0.00,500,Rain,light rain,10d
